In [ ]:
import numpy as np
import nltk

from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000

from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer

from sklearn.utils import shuffle

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\JheWei\.conda\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\JheWei\AppData\Local\Temp\ipykernel_1664200\4168014651.py", line 8, in <module>
    from keras.preprocessing.sequence import pad_sequences
  File "C:\Users\JheWei\.conda\envs\nlp\lib\site-packages\keras\__init__.py", line 20, in <module>
    from keras import distribute
  File "C:\Users\JheWei\.conda\envs\nlp\lib\site-packages\keras\distribute\__init__.py", line 18, in <module>
    from keras.distribute import sidecar_evaluator
  File "C:\Users\JheWei\.conda\envs\nlp\lib\site-packages\keras\distribute\sidecar_evaluator.py", line 22, in <module>
    from keras.optimizers.optimizer_experimental import (
  File "C:\Users\JheWei\.conda\envs\nlp\lib\site-packages\keras\optimizers\__init__.py", line 25, in <module>
    from keras import backend
  File "C:\Users\JheWei\.cond

In [2]:
#nltk.download('treebank')
#nltk.download('brown')
#nltk.download('conll2000')

treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')
tagged_sentences = treebank_corpus + brown_corpus + conll_corpus

In [3]:
print('data size:',len(tagged_sentences))
nltk.help.upenn_tagset()
tagged_sentences[7]

data size: 72202
$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
I

[('A', 'DET'),
 ('Lorillard', 'NOUN'),
 ('spokewoman', 'NOUN'),
 ('said', 'VERB'),
 (',', '.'),
 ('``', '.'),
 ('This', 'DET'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('old', 'ADJ'),
 ('story', 'NOUN'),
 ('.', '.')]

In [4]:
words = []
tags = []

for sentence in tagged_sentences:
    X_sentence = []
    Y_sentence = []
    for entity in sentence:         
        X_sentence.append(entity[0])
        Y_sentence.append(entity[1])
        
    words.append(X_sentence)
    tags.append(Y_sentence)

In [5]:
print('sample words: ', words[0])
print('sample tags: ', tags[0])

sample words:  ['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
sample tags:  ['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']


In [6]:
print("Length of first input sequence  : {}".format(len(words[0])))
print("Length of first output sequence : {}".format(len(tags[0])))

Length of first input sequence  : 18
Length of first output sequence : 18


In [7]:
import numpy as np
def set_vocab(words):
    set_words = list(set([t1 for t2 in words for t1 in t2]))
    words_index = np.array(range(len(set_words))) + 1
    word_to_index = dict(zip(set_words, words_index))
    index_to_word = dict(zip(words_index, set_words))
    return word_to_index,index_to_word

In [8]:
word_to_index,index_to_word = set_vocab(words)
tag_to_index,index_to_tag = set_vocab(tags)
print(tag_to_index)
print(word_to_index)

{'ADV': 1, 'NOUN': 2, '.': 3, 'ADP': 4, 'PRT': 5, 'PRON': 6, 'X': 7, 'CONJ': 8, 'ADJ': 9, 'DET': 10, 'NUM': 11, 'VERB': 12}
{'masterpiece': 1, 'pastry-lined': 2, 'partake': 3, 'forgo': 4, 'Theatre': 5, 'Discours': 6, 'ultracentrifugation': 7, 'CD-ROM': 8, 'astound': 9, 'gooshey': 10, 'chose': 11, 'Emanuele': 12, "Rembrandt's": 13, 'underwent': 14, 'Benson': 15, 'conformed': 16, 'issuing': 17, 'Macmillan\\/McGraw': 18, 'dines': 19, 'radiating': 20, 'Activities': 21, '580': 22, "master's": 23, 'age-and-sex': 24, 'poultice': 25, 'Loewi': 26, '0.4': 27, '102.625': 28, 'chaw': 29, 'sentenced': 30, 'furthering': 31, '8.59': 32, 'zeros': 33, 'Boyce': 34, 'Academy': 35, 'Davenport': 36, 'weighs': 37, 'rose-pink': 38, 'reselling': 39, 'separators': 40, 'reluctantly': 41, 'coccidiosis': 42, 'forbad': 43, '143,178': 44, 'Keehn': 45, 'Isabel': 46, "Lew's": 47, 'Bergson': 48, 'Renfrew': 49, 'quasi-recitative': 50, 'receiver': 51, 'terraced': 52, '$3.11': 53, 'checking-account': 54, 'Philippine': 55

In [9]:
word_vocab_size = list(index_to_word)[-1] +1
tag_vocab_size = list(index_to_tag)[-1] +1
print('word vocabulary size:', word_vocab_size)
print('tag vocabulary size:', tag_vocab_size)

word vocabulary size: 67068
tag vocabulary size: 13


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split([' '.join(w) for w in words], [' '.join(w) for w in tags], test_size=0.2)

In [11]:
print('Training data size: %d' %len(X_train))
print('Testing data size: %d' %len(X_test))

Training data size: 57761
Testing data size: 14441


In [12]:
import tensorflow as tf
train_tfdata = tf.data.Dataset.from_tensor_slices((X_train,y_train))
test_tfdata = tf.data.Dataset.from_tensor_slices((X_test,y_test))

In [13]:
x = iter(train_tfdata)
tmp_inp =next(x)
print(tmp_inp)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Thus cortico-fugal discharges induced by topical application of strychnine to a minute area in the neocortex summate with spikes present in the hypothalamus and cause increased convulsive discharges .'>, <tf.Tensor: shape=(), dtype=string, numpy=b'ADV ADJ NOUN VERB ADP ADJ NOUN ADP NOUN ADP DET ADJ NOUN ADP DET NOUN VERB ADP NOUN ADV ADP DET NOUN CONJ VERB VERB ADJ NOUN .'>)


In [14]:
def encode(word,tag):
    word = [word_to_index[t] for t in word.numpy().decode().split(' ')]
    tag = [tag_to_index[t] for t in tag.numpy().decode().split(' ')]
    return word, tag

def tf_encode(word, tag):
    return tf.py_function(encode, [word,tag], [tf.int32, tf.int32])

In [15]:
buffer_size =320
batch_size = 32
padded_shapes = (tf.TensorShape([None]), tf.TensorShape([None]))
train_generator = train_tfdata.map(tf_encode, num_parallel_calls=tf.data.experimental.AUTOTUNE).cache().shuffle(buffer_size).padded_batch(batch_size,padded_shapes=padded_shapes).repeat()
test_generator = test_tfdata.map(tf_encode, num_parallel_calls=tf.data.experimental.AUTOTUNE).padded_batch(batch_size,padded_shapes=padded_shapes)

x = iter(train_generator)
tmp_inp =next(x)
print(tmp_inp)

(<tf.Tensor: shape=(32, 70), dtype=int32, numpy=
array([[ 5966, 34101, 44940, ...,     0,     0,     0],
       [47323, 30447, 26826, ...,     0,     0,     0],
       [ 5363,  2470, 37576, ...,     0,     0,     0],
       ...,
       [62762, 57387, 19359, ...,     0,     0,     0],
       [13095, 39006,  9208, ...,     0,     0,     0],
       [27878,  2535, 29917, ...,     0,     0,     0]])>, <tf.Tensor: shape=(32, 70), dtype=int32, numpy=
array([[ 9,  9,  9, ...,  0,  0,  0],
       [ 7,  1,  8, ...,  0,  0,  0],
       [11,  6, 10, ...,  0,  0,  0],
       ...,
       [ 1, 12,  9, ...,  0,  0,  0],
       [10, 11,  1, ...,  0,  0,  0],
       [ 2,  8, 12, ...,  0,  0,  0]])>)


In [16]:
class postag_rnn(tf.keras.Model):
    def __init__(self, embedding_size, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(input_dim=word_vocab_size, output_dim=embedding_size)
        self.rnn = tf.keras.layers.SimpleRNN(rnn_units,return_sequences=True)
        output_layer =tf.keras.layers.Dense(units=tag_vocab_size,activation='softmax')
        self.timedistributed = tf.keras.layers.TimeDistributed(output_layer)
        
    def call(self,x):
        embedded = self.embedding(x)
        hidden_states = self.rnn(embedded)
        outputs = self.timedistributed(hidden_states)
        
        return outputs

In [17]:
embedding_size = 256
rnn_units = 512

model = postag_rnn(embedding_size,rnn_units)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(train_generator,epochs=2,validation_data=test_generator, steps_per_epoch=len(X_train) // batch_size+1)

loss ,accuracy = model.evaluate(test_generator)
print("test dataset's accuracy: {:.2f}".format(accuracy))

Epoch 1/2
1806/1806 [==============================] - 266s 147ms/step - loss: 0.2165 - accuracy: 0.9409 - val_loss: 0.0568 - val_accuracy: 0.9811
Epoch 2/2
452/452 [==============================] - 8s 18ms/step - loss: 0.0547 - accuracy: 0.9819
test dataset's accuracy: 0.98


In [25]:
class postag_lstm(tf.keras.Model):
    def __init__(self, embedding_size, lstm_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(input_dim=word_vocab_size, output_dim=embedding_size)
        self.lstm = tf.keras.layers.LSTM(lstm_units,return_sequences=True)
        output_layer =tf.keras.layers.Dense(units=tag_vocab_size,activation='softmax')
        self.timedistributed = tf.keras.layers.TimeDistributed(output_layer)
        
    def call(self,x):
        embedded = self.embedding(x)
        hidden_states = self.lstm(embedded)
        outputs = self.timedistributed(hidden_states)
        
        return outputs

In [26]:
embedding_size = 256
rnn_units = 512

model = postag_lstm(embedding_size,rnn_units)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(train_generator,epochs=2,validation_data=test_generator, steps_per_epoch=len(X_train) // batch_size+1)

loss ,accuracy = model.evaluate(test_generator)
print("test dataset's accuracy: {:.2f}".format(accuracy))

Epoch 1/2
1806/1806 [==============================] - 686s 379ms/step - loss: 0.1358 - accuracy: 0.9572 - val_loss: 0.0510 - val_accuracy: 0.9828
Epoch 2/2
452/452 [==============================] - 53s 118ms/step - loss: 0.0470 - accuracy: 0.9846
test dataset's accuracy: 0.98


In [27]:
testing_preds= list()
testing_true= list()

for test in test_generator:
    words, tags =test
    testing_pred = model.predict(words)
    testing_pred_index = np.argmax(testing_pred, axis=-1)
    
    for i in range(len(tags)):
        testing_preds.append([p for p in testing_pred_index[i] if p != 0])
        testing_true.append([p for p in tags[i].numpy() if p != 0])


print_index=5
word = X_test[print_index]
pred = testing_preds[print_index]
true = testing_true[print_index]

pred_tag = [index_to_tag[t] for t in pred]
true_tag = [index_to_tag[t] for t in true]

print('Input words: \n', word)
print('Prediction: \n', pred_tag)
print('True: \n',true_tag)

Input words: 
 It is a competence and expertise that some brokers themselves , overwhelmed by all the new things being dreamed up for them to peddle , do n't feel .
Prediction: 
 ['PRON', 'VERB', 'DET', 'NOUN', 'CONJ', 'NOUN', 'PRON', 'DET', 'NOUN', 'PRON', '.', 'VERB', 'ADP', 'PRT', 'DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'PRT', 'ADP', 'PRON', 'PRT', 'VERB', '.', 'VERB', 'ADV', 'VERB', '.']
True: 
 ['PRON', 'VERB', 'DET', 'NOUN', 'CONJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'PRON', '.', 'VERB', 'ADP', 'DET', 'DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'ADP', 'ADP', 'PRON', 'PRT', 'VERB', '.', 'VERB', 'ADV', 'VERB', '.']
